# 00 Import Libraries

In [1]:
from openai import OpenAI
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask, render_template, request

# 01 Functions

In [2]:
# FUNCTION TO OPEN FILES
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()
    
# FUNCTION TO CALCULATE SIMILARITY AND RETURN BEST MATCH
def similarity(question_embedding):
    
    # LOAD BOOKS FROM JSON FILE
    with open('json/books_info_embeddings.json', 'r') as f:
        products = json.load(f)
     
    # CALCULATE SIMILARITY BETWEEN QUESTION AND BOOKS
    best_match = None
    best_similarity = -1
    
    # LOOP THRU ALL BOOKS
    for product in products:

        product_embedding = np.array(product['text_embedding'])
        similarity = cosine_similarity([question_embedding], [product_embedding])[0][0]
        
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = product
    
    # PREPARE SYSTEM MESSAGE WITH DETAILS ABOUT BEST MATCHING BOOK
    best_match_system = f"""BOOK: {best_match['book_title']}
    CATEGORY: {best_match['book_category']}
    DESCRIPTION: {best_match['book_description']}
    RATING: {best_match['book_rating']}
    PRICE: {best_match['book_price']}
    AVAILABILITY: {best_match['book_availability']}"""
    
    return best_match_system

# 02 Credentials

In [3]:
# LOAD OPENAI CREDENTIALS AND SYSTEM MESSAGE
client = OpenAI(api_key=open_file('keys/openaiapikey.txt'))
system = open_file('prompts/system.txt')

# 03 Flask App

In [4]:
# FLASK APP
app = Flask(__name__)

# APP
@app.route("/")
def index():

    return render_template("index.html")

@app.route('/get', methods=['POST'])
def get_bot_response():
    
    # READ USER INPUT
    user_input = request.form['user_input']
    
    # RECOMMENDATIONS FÜR USER INPUT
    question_embedding = client.embeddings.create(input=user_input, model="text-embedding-3-small").data[0].embedding
    best_match_system = similarity(question_embedding)
    
    # RETURN RESPONSE
    return render_template("index.html", response_area=best_match_system)

# 04 Run the App

In [5]:
port = 8083
app.run(host='0.0.0.0', port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8083
 * Running on http://192.168.41.162:8083
Press CTRL+C to quit
127.0.0.1 - - [08/Mar/2024 16:45:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 16:46:03] "POST /get HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 16:47:02] "POST /get HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 16:47:15] "POST /get HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 16:47:30] "POST /get HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 16:47:37] "GET /get HTTP/1.1" 405 -
127.0.0.1 - - [08/Mar/2024 16:47:39] "GET / HTTP/1.1" 200 -
